## Data Collection

The dataset that used was taken from Humanitarian Data Exchange (HDX). It is the World Food Programme Price Database which contains data regarding food prices in the Philippines. The data in this dataset spans from 2000 - 2023 garnering 184,714 entries (rows). The dataset is in the file named `wfp_food_prices_phl.csv` in the root folder but it was also made publicly available through the following:

Link: https://data.humdata.org/dataset/wfp-food-prices-for-philippines?force_layout=desktop.

#### The dataset was imported as follows:

In [1]:
#import libraries
import pandas as pd
import numpy as np

#import the dataset
data = pd.read_csv('wfp_food_prices_phl.csv')
data.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,inflation
0,1/15/00,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,Maize flour (yellow),KG,actual,Retail,PHP,15.00,0.3717,3.98
1,1/15/00,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (milled, superior)",KG,actual,Retail,PHP,20.00,0.4957,3.98
2,1/15/00,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (milled, superior)",KG,actual,Wholesale,PHP,18.35,0.4548,3.98
3,1/15/00,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (regular, milled)",KG,actual,Retail,PHP,18.00,0.4461,3.98
4,1/15/00,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (regular, milled)",KG,actual,Wholesale,PHP,16.35,0.4052,3.98


#### Inspecting the columns of the dataset:

In [2]:
#check the columns of the dataset
print("Columns: ", data.columns)
#check the shape of the dataset
print("Data Shape: ",data.shape)

Columns:  Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice', 'inflation'],
      dtype='object')
Data Shape:  (184714, 15)


## Data preprocessing
Perform data preprocessing and identify columns to be used. You may or may not use all the columns. Prepare the features and target data.
Prepare the train and test data.

#### Inspecting the datatype of each column:

In [3]:
#check the datatypes of each column
data.dtypes

date          object
admin1        object
admin2        object
market        object
latitude     float64
longitude    float64
category      object
commodity     object
unit          object
priceflag     object
pricetype     object
currency      object
price        float64
usdprice     float64
inflation    float64
dtype: object

We will fix the datatypes and convert them to the proper datatypes but in this case we will only convert the date. For the other categorical columns, we will just get dummy values in order to fit them to the model.

In [4]:
#Change the datatype of date and get the year and month
#Add a year and month column which will be used for our regression model
data['date'] = data['date'].astype('datetime64[ns]')
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [5]:
#Drop all the null values and the rows with price == 0 
#since there are null or 0 price in the dataset
#We need to remove them because those are incomplete data
data.dropna(inplace= True)
data.drop(data.loc[data['price']==0].index, inplace=True)

In [6]:
#Drop the unnecessary columns from the dataset
data = data.drop(['date', 'admin1','admin2','market','category', 'currency', 'unit'
        ,'usdprice'], axis='columns')

#Get dummy values for the categorical columns
data = pd.get_dummies(data=data)

#Divide the data into test and training sets
X = data.drop(['price'], axis='columns')
y = data['price'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=26)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
display(X_train)

(109930, 84)
(27483, 84)
(109930,)
(27483,)


,latitude,longitude,inflation,year,month,commodity_Anchovies,commodity_Bananas (lakatan),commodity_Bananas (latundan),commodity_Bananas (saba),"commodity_Beans (green, fresh)",...,commodity_Sweet potatoes,commodity_Taro,commodity_Tomatoes,commodity_Water spinach,priceflag_actual,"priceflag_actual,aggregate",priceflag_aggregate,pricetype_Farm Gate,pricetype_Retail,pricetype_Wholesale
15312,16.016667,120.233333,3.03,2012,11,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
51810,14.604167,120.982222,2.39,2020,9,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
34435,16.486093,121.146518,2.39,2020,5,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
35778,11.706772,122.370090,2.39,2020,5,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
49991,8.040911,123.799419,2.39,2020,8,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73408,13.146926,123.750464,3.93,2021,2,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
137475,10.667360,122.946930,5.80,2022,7,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
77256,10.132101,124.834680,3.93,2021,3,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
59971,13.137222,123.734444,2.39,2020,10,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


The following columns were removed:

1) Date - Since there is already a month and year columns which we extracted from the date column.
2) Admin1, Admin2, Market - We will use the longitude and latitude for the location so that the locations not included can still be estimated based on the longitude and latitude.
3) Category - We will not use this since it will decreases the accuracy of the model because there are many commodities in a category and all of them have different prices.
4) Unit, Currency, Usd Price - All of these columns are irrelevant for the model since unit is just the measurement, all currencies are Philippine peso, and USD price is just the converted price.

## Model
### Performing linear regression on the dataset.
Using three types of regression:

In [7]:
#Train the model using the training set
from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV
regressor1 = LinearRegression()
regressor2 = RidgeCV(alphas=np.logspace(-3, 3, 50))
regressor3 = ElasticNetCV()

regressor1.fit(X_train, y_train)
regressor2.fit(X_train, y_train)
regressor3.fit(X_train, y_train)
                     
#Use the model on the testing set
test_predictions1 = regressor1.predict(X_test)
test_predictions2 = regressor2.predict(X_test)
test_predictions3 = regressor3.predict(X_test)

### Results.

#### Using LinearRegression (Ordinary Least Squares) and default parameters:

In [8]:
#get the MAE,MSE,RMSE, and R2 values to evaluate the model 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

MAE = mean_absolute_error(y_test,test_predictions1)
MSE = mean_squared_error(y_test,test_predictions1)
RMSE = np.sqrt(MSE)
r2 = r2_score(y_test,test_predictions1)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":test_predictions1})
display(comparison_df)

print(f"MAE = {MAE}\nMSE = {MSE}\nRMSE = {RMSE}\nr2 = {r2}")

,Actual,Predicted
0,238.36,236.942825
1,40.00,49.252274
2,37.50,35.768753
3,42.77,23.842270
4,138.00,205.297989
...,...,...
27478,62.00,47.890579
27479,102.75,141.857712
27480,207.50,187.664993
27481,36.74,27.844589


MAE = 24.16171182575329
MSE = 1476.6919285868055
RMSE = 38.42774946034188
r2 = 0.871490385565866


#### Using Ridge Regression (with built-in cross-validation) and alpha values in np.logspace(-3, 3, 50) :

In [9]:
print(f"alpha = {regressor2.alpha_}")

MAE = mean_absolute_error(y_test,test_predictions2)
MSE = mean_squared_error(y_test,test_predictions2)
RMSE = np.sqrt(MSE)
r2 = r2_score(y_test,test_predictions2)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":test_predictions2})
display(comparison_df)

print(f"MAE = {MAE}\nMSE = {MSE}\nRMSE = {RMSE}\nr2 = {r2}")

alpha = 0.15998587196060574


,Actual,Predicted
0,238.36,236.936802
1,40.00,49.256008
2,37.50,35.783173
3,42.77,23.848158
4,138.00,205.291324
...,...,...
27478,62.00,47.958698
27479,102.75,141.857657
27480,207.50,187.660962
27481,36.74,27.872759


MAE = 24.161873197422274
MSE = 1476.672933169809
RMSE = 38.427502301994714
r2 = 0.8714920386484535


#### Using Elastic Net Regression (with cross-validation) and deafult parameters (5-fold cross validation):

In [10]:
print(f"alpha = {regressor3.alpha_}")

MAE = mean_absolute_error(y_test,test_predictions3)
MSE = mean_squared_error(y_test,test_predictions3)
RMSE = np.sqrt(MSE)
r2 = r2_score(y_test,test_predictions3)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":test_predictions3})
display(comparison_df)

print(f"MAE = {MAE}\nMSE = {MSE}\nRMSE = {RMSE}\nr2 = {r2}")

alpha = 0.21987709675486372


,Actual,Predicted
0,238.36,124.420421
1,40.00,92.619586
2,37.50,118.236110
3,42.77,97.087158
4,138.00,102.217260
...,...,...
27478,62.00,105.505210
27479,102.75,133.096068
27480,207.50,141.976584
27481,36.74,109.601797


MAE = 65.99080589970156
MSE = 8450.866881518366
RMSE = 91.92859664717159
r2 = 0.2645604519438006


#### OLS vs Ridge vs ElasticNet
RidgeCV yung best so far pero not by much. mababa ata score ng elasticCV kasi masyado aggressive sa regularization nazzero out yung ibang important variables like commodity.

### Extending the Linear Model using Polynomial Features

#### Preprocessing
explain why polynomial features here

In [11]:
data['month2'] = data['month']**2 # add degree of freedom
data['longlat'] = data['longitude']*data['latitude'] # long-lat relationship
data['inflyr'] = data['year']*data['inflation'] # inflation-year relationship
data['locyr'] = data['inflyr']*data['longlat'] # location-year relationship
data['locmth'] = data['month']*data['month2']*data['longlat'] # location-month relationship

#divide the data into test and training sets
X = data.drop(['price'], axis='columns')
y = data['price'].values

# add interactions for each commodity
for i in X.columns:
    if i not in ['locyr','locmth','inflation','year','inflyr','longlat','latitude','longitude','month','month2','priceflag_actual','priceflag_actual,aggregate','priceflag_aggregate','pricetype_Farm Gate','pricetype_Retail','pricetype_Wholesale']:
        for j in ['locyr','locmth','inflation','year','inflyr','longlat','latitude','longitude','month','month2','priceflag_actual','priceflag_actual,aggregate','priceflag_aggregate','pricetype_Farm Gate','pricetype_Retail','pricetype_Wholesale']:
            X=pd.concat((X,(X[i]*X[j]).rename(i+j)),axis=1)
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=26)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
display(X_train)

(109930, 1257)
(27483, 1257)
(109930,)
(27483,)


,latitude,longitude,inflation,year,month,commodity_Anchovies,commodity_Bananas (lakatan),commodity_Bananas (latundan),commodity_Bananas (saba),"commodity_Beans (green, fresh)",...,commodity_Water spinachlatitude,commodity_Water spinachlongitude,commodity_Water spinachmonth,commodity_Water spinachmonth2,commodity_Water spinachpriceflag_actual,"commodity_Water spinachpriceflag_actual,aggregate",commodity_Water spinachpriceflag_aggregate,commodity_Water spinachpricetype_Farm Gate,commodity_Water spinachpricetype_Retail,commodity_Water spinachpricetype_Wholesale
15312,16.016667,120.233333,3.03,2012,11,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
51810,14.604167,120.982222,2.39,2020,9,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
34435,16.486093,121.146518,2.39,2020,5,0,0,0,0,1,...,0.0,0.0,0,0,0,0,0,0,0,0
35778,11.706772,122.370090,2.39,2020,5,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
49991,8.040911,123.799419,2.39,2020,8,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73408,13.146926,123.750464,3.93,2021,2,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
137475,10.667360,122.946930,5.80,2022,7,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
77256,10.132101,124.834680,3.93,2021,3,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
59971,13.137222,123.734444,2.39,2020,10,0,0,0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0


#### Performing Linear Regression

In [12]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
display(regressor.coef_)

test_predictions = regressor.predict(X_test)

#get the MAE,MSE,RMSE, and R2 values to evaluate the model 
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
r2 = r2_score(y_test,test_predictions)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":test_predictions})
display(comparison_df)

print(f"MAE = {MAE}\nMSE = {MSE}\nRMSE = {RMSE}\nr2 = {r2}")

array([1.02378276e+06, 9.75368716e+01, 1.99895685e+03, ...,
       0.00000000e+00, 6.70589912e+03, 0.00000000e+00])

,Actual,Predicted
0,238.36,219.762433
1,40.00,50.416495
2,37.50,39.306686
3,42.77,29.652583
4,138.00,162.966873
...,...,...
27478,62.00,56.670491
27479,102.75,169.109329
27480,207.50,202.983438
27481,36.74,34.814321


MAE = 17.119564308671325
MSE = 908.9223205379819
RMSE = 30.148338603279317
r2 = 0.9209007276997193


#### Using Ridge Regression:

In [13]:
from sklearn.linear_model import Ridge
regressor = Ridge()
regressor.fit(X_train, y_train)

test_predictions = regressor.predict(X_test)

#get the MAE,MSE,RMSE, and R2 values to evaluate the model 
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
r2 = r2_score(y_test,test_predictions)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":test_predictions})
display(comparison_df)

print(f"MAE = {MAE}\nMSE = {MSE}\nRMSE = {RMSE}\nr2 = {r2}")

,Actual,Predicted
0,238.36,215.286777
1,40.00,49.918975
2,37.50,37.122897
3,42.77,31.598646
4,138.00,153.151989
...,...,...
27478,62.00,56.855291
27479,102.75,159.734746
27480,207.50,198.454571
27481,36.74,32.129576


MAE = 18.28534051619844
MSE = 972.9907438306825
RMSE = 31.192799551029122
r2 = 0.9153251514976964


## Results

### Summary of metrics
summarize models here (best ata yung ordinary least squares regression with polynomial features)

### Model Comparison
compare sa other studies